In [1]:
#ignition delay for RMG model of propane 
import pandas as pd
import numpy as np
import time
import cantera as ct
import sys 

In [2]:
print(f"Runnning Cantera version: {ct.__version__}")

#constants
pressure = 101325 #Pascals
mole_frac_dict = {"C3H8(1)": 1, "O2(2)": 5, "Ar":94} #this technically sets equivalence ratio to 1 
#mole_frac_dict_eduardos = {"C3H8(1)": 1, "O2(2)": 6, "Ar":93}

def ignition_delay(states, species): 
    """
    This function computes the ignition delay from the occurence of the peak in species' concentration
    """
    # pd = states.to_pandas()
    # print(pd)
    i_ign = states(species).Y.argmax()
    return states.t[i_ign]

def gas_conditions(temp): 
    """
    define gas conditions
    """
    gas.TPX = temp, pressure, mole_frac_dict

Runnning Cantera version: 2.5.1


In [3]:
#to save data
taus = []


reference_species = "OH(6)"

x_axis_vals= list(np.linspace(0.55, 0.8, 20))


for i in x_axis_vals: 
        
    gas = ct.Solution("./cantera/chem_annotated.yaml")
    temp= 1000/i
    gas_conditions(temp)
    
    # Create a batch reactor object and add it to a reactor network
    # In this example, the batch reactor will be the only reactor in the network
    r = ct.IdealGasReactor(contents=gas, name="Batch Reactor")
    reactor_network = ct.ReactorNet([r])
    
    
    # use the above list to create a DataFrame
    time_history = ct.SolutionArray(gas, extra="t")

    t0 = time.time()
    
    # This is a starting estimate. If you do not get an ignition within this time, increase it
    estimated_ignition_delay_time = 0.4
    t = 0

    counter = 1
    while t < estimated_ignition_delay_time:
        t = reactor_network.step()
        if not counter % 10:
            # We will save only every 10th value. Otherwise, this takes too long
            # Note that the species concentrations are mass fractions
            time_history.append(r.thermo.state, t=t)
        counter += 1

    # We will use the 'OH(6)' species to compute the ignition delay
    tau = ignition_delay(time_history, reference_species)

    # Toc
    t1 = time.time()

    print(f"Computed Ignition Delay: {tau:.3e} seconds. Took {t1-t0:3.2f}s to compute")

    #save to large data array
    taus.append(tau)
    
    # If you want to save all the data - molefractions, temperature, pressure, etc
    #time_history.to_pandas().to_csv("time_history.csv")


Computed Ignition Delay: 1.296e-04 seconds. Took 0.64s to compute
Computed Ignition Delay: 1.295e-04 seconds. Took 0.64s to compute
Computed Ignition Delay: 1.348e-04 seconds. Took 0.49s to compute
Computed Ignition Delay: 1.414e-04 seconds. Took 0.51s to compute
Computed Ignition Delay: 1.550e-04 seconds. Took 0.57s to compute
Computed Ignition Delay: 1.682e-04 seconds. Took 0.50s to compute
Computed Ignition Delay: 1.888e-04 seconds. Took 0.56s to compute
Computed Ignition Delay: 2.258e-04 seconds. Took 0.58s to compute
Computed Ignition Delay: 2.709e-04 seconds. Took 0.60s to compute
Computed Ignition Delay: 3.472e-04 seconds. Took 0.68s to compute
Computed Ignition Delay: 4.466e-04 seconds. Took 0.62s to compute
Computed Ignition Delay: 5.969e-04 seconds. Took 0.65s to compute
Computed Ignition Delay: 8.167e-04 seconds. Took 0.61s to compute
Computed Ignition Delay: 1.145e-03 seconds. Took 0.66s to compute
Computed Ignition Delay: 1.620e-03 seconds. Took 0.68s to compute
Computed I

In [48]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays_eds_stoic.csv', index=False)

In [4]:
#save to csv file 
dataframe = pd.DataFrame([x_axis_vals, taus]) 
dataframe.to_csv('C3H8_ignition_delays.csv', index=False)

In [6]:
nora_df = pd.read_csv('/work/westgroup/nora/Code/projects/PFAS/models/C3H8_CH2F2/C3H8/C3H8_sevys_settings /C3H8_ignition_delays.csv')
nora_df.iloc[1].values

array([0.00012963, 0.00012953, 0.00013485, 0.00014138, 0.00015504,
       0.00016817, 0.00018882, 0.00022575, 0.00027094, 0.00034722,
       0.00044664, 0.00059694, 0.00081667, 0.0011449 , 0.00161999,
       0.00233402, 0.00338168, 0.00492469, 0.00718034, 0.01043739])